Carrega as bibliotecas e testa a conexão com banco de dados PostGreeSQL

In [1]:
import connection as cn
from sqlalchemy import text, inspect, Engine
import pandas as pd

cn.connection_test()

✅ Sucesso: Conexão estabelecida com o banco de dados.


Lista as tabelas do banco de dados

In [4]:
# 1. Configurações iniciais
con = cn.get_session()
schema_name = 'analytics'
dados_tabelas = []

# 2. Busca otimizada dos nomes das tabelas e criação do inspector
# Passamos con.bind (a Engine) para inspect()
inspector = inspect(con.bind) 
tabelas = inspector.get_table_names(schema=schema_name)

# 3. Itera sobre as tabelas, obtém contagem E colunas
for tabela in tabelas:
    # 3.1. Obtém a lista de colunas
    try:
        # Usa o inspector para obter a lista de dicionários de colunas
        colunas_info = inspector.get_columns(tabela, schema=schema_name)
        
        # Extrai apenas o nome ('name') de cada coluna e cria a lista [col1, col2, ...]
        lista_colunas = [col['name'] for col in colunas_info]
        
    except Exception as e:
        # Em caso de erro ao inspecionar colunas (ex: tabela inacessível)
        lista_colunas = f"ERRO INSPECT: {e.__class__.__name__}"


    # 3.2. Obtém a contagem de registros (igual ao código original)
    try:
        sql_count = text(f"SELECT COUNT(*) FROM {schema_name}.{tabela};")
        count = con.execute(sql_count).scalar_one()

    except Exception as e:
        count = f"ERRO COUNT: {e.__class__.__name__}"
        
        
    # 3.3. Adiciona todos os dados (incluindo a nova coluna) à lista
    dados_tabelas.append({
        'tablename': tabela,
        'record_count': count,
        'columns_list': lista_colunas  # << NOVA COLUNA AQUI
    })

# 4. Cria o DataFrame e imprime
df_tabelas = pd.DataFrame(dados_tabelas)
df_tabelas.to_csv('tabelas_colunas.csv')
df_tabelas

,tablename,record_count,columns_list
0,df_orders,99441,"[order_id, customer_id, order_status, order_pu..."
1,dim_customer,99441,"[customer_sk, customer_id, effective_from, eff..."
2,fact_sales,112650,"[sale_sk, order_id, customer_sk, product_sk, s..."
3,dim_product,32951,"[product_sk, product_id]"
4,dim_seller,3095,"[seller_sk, seller_id]"
5,dim_date,773,"[date_sk, full_date, year, month, day]"
6,df_items,112650,"[order_id, order_item_id, product_id, seller_i..."


Executa consultas SQL e retorna resultado em um dataframe PANDAS

In [32]:
consulta = text("""
SELECT
    ds.seller_id,
    ds.seller_sk,
    SUM(fs.price + fs.freight_value) AS total_valor_venda
FROM
    fact_sales fs
JOIN
    dim_seller ds ON fs.seller_sk = ds.seller_sk
GROUP BY
    ds.seller_id,
    ds.seller_sk
ORDER BY
    total_valor_venda DESC
LIMIT 10;
""")
df = pd.read_sql(sql=consulta, con=cn.get_engine())
df.head()

,seller_id,seller_sk,total_valor_venda
0,4869f7a5dfa277a7dca6462dcf3b52b2,518,249640.70
1,7c67e1448b00f6e969d365cea6b010ab,178,239536.44
2,53243585a1d6dc2643021fd1853d8905,3037,235856.68
3,4a3ca9315b744ce9f8e9374361493884,60,235539.96
4,fa1c13f2614d7b5c4749cbc52fecda94,1898,204084.73
